In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K

from PIL import Image

In [ ]:
# Original width: 600
# IM_WIDTH = 400 # Keep width = 400 for InceptionV4
IM_WIDTH = 300 # Keep width = 300 for ResNet50

# Original height: 450
# IM_HEIGHT = 300 # Keep height = 300 for InceptionV4
IM_HEIGHT = 225 # Keep height = 225 for ResNet50

BATCH_SIZE = 16

SEED = 429347

data_path = '/home/ubuntu/Data/Skin/df_vasc_others/'

global_mean = np.load(data_path + 'mean_image_df_vasc_others.npz')['image'].astype(np.uint8)
mean_image = Image.fromarray(global_mean).resize((IM_WIDTH, IM_HEIGHT))
global_mean = np.asarray(mean_image, dtype=np.float32)
np.random.seed(SEED)

In [ ]:
def preprocess_image(image):
    return image - global_mean

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import compute_class_weight

train_img = ImageDataGenerator(preprocessing_function=preprocess_image, rescale=1./255)

img_gen = ImageDataGenerator(preprocessing_function=preprocess_image, rescale=1./255)

train_gen = train_img.flow_from_directory(data_path + 'train', 
                                        target_size=(IM_HEIGHT, IM_WIDTH), 
                                        class_mode='categorical', 
                                        shuffle=True, 
                                        batch_size=BATCH_SIZE, 
                                        classes=['DF', 'VASC', 'OTHERS'], 
                                          seed=SEED
                                       )

print('Training Set Generator done.\n')
val_gen = img_gen.flow_from_directory(data_path + 'val', 
                                      target_size=(IM_HEIGHT, IM_WIDTH), 
                                      class_mode='categorical', 
                                      shuffle=True, 
                                      batch_size=BATCH_SIZE, 
                                      classes=['DF', 'VASC', 'OTHERS'],
                                      seed=SEED
                                     )

print('Validation Set Generator done.\n')
test_gen = img_gen.flow_from_directory(data_path + 'test', 
                                       target_size=(IM_HEIGHT, IM_WIDTH), 
                                       class_mode='categorical', 
                                       shuffle=False, 
                                       batch_size=BATCH_SIZE, 
                                       classes=['DF', 'VASC', 'OTHERS'],
                                       seed=SEED
                                      )
print('Test Set Generator done.\n')
y_train = train_gen.classes
weights = compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {i: weights[i] for i in range(len(weights))}

print(class_weights)
print(train_gen.class_indices)

In [ ]:
num_hidden_units = 512
model_name = 'inceptionv3'
# model_name = 'resnet50'
# model_name = 'inceptionv4'
# model_name = 'vgg16'
# model_name = 'densenet'

train_type = 'full_train'
data = 'df_vasc_others'

filename = data + '_' + model_name + '_' + str(num_hidden_units) + '_' + train_type + '.h5'
print(filename)

In [ ]:
from keras.applications.inception_v3 import InceptionV3
# from keras.applications.resnet50 import ResNet50
# from keras.applications.vgg16 import VGG16
# import inception_v4
from keras import Model
from keras.layers import (Dense, Activation, Dropout, 
                          BatchNormalization, Flatten, GlobalAveragePooling2D)

inc = InceptionV3(include_top=False, input_shape=(IM_HEIGHT, IM_WIDTH, 3), weights='imagenet')
# inc = ResNet50(include_top=False, input_shape=(IM_HEIGHT, IM_WIDTH, 3), weights='imagenet')
# inc = VGG16(include_top=False, input_shape=(IM_HEIGHT, IM_WIDTH, 3), weights='imagenet')
# inc = inception_v4.create_model(weights=None, include_top=False, input_shape=(IM_HEIGHT, IM_WIDTH, 3))
# inc.load_weights('/home/ubuntu/Notebooks/Models/inception-v4_weights_tf_dim_ordering_tf_kernels_notop.h5')
x = inc.output
# x = GlobalAveragePooling2D()(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(num_hidden_units)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.2)(x)
out = Dense(3, activation='softmax')(x)

model = Model(inputs=inc.input, outputs=out)

# last_layer_weights = model.layers[-1].get_weights()
# last_layer_weights[1][:2] += 1.
# last_layer_weights[1][2:] -= 1.

# model.layers[-1].set_weights(last_layer_weights)

# print(model.layers[-1].get_weights()[1])

for layer in inc.layers:
    layer.trainable = False

In [ ]:
print(model.summary())

In [ ]:
def f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return ( 2*(precision * recall) / (precision + recall + K.epsilon()) )

In [ ]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return (true_positives / (predicted_positives + K.epsilon()))

In [ ]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return (true_positives / (possible_positives + K.epsilon()))

In [ ]:
from keras.optimizers import Adam

model.compile(loss='categorical_crossentropy', optimizer=Adam(5e-4))

_ = model.fit_generator(train_gen, 
                        steps_per_epoch=train_gen.samples//BATCH_SIZE + 1, 
                        validation_data=val_gen, 
                        validation_steps=val_gen.samples//BATCH_SIZE + 1, 
                        shuffle=True, 
                        epochs=1, 
                        class_weight=class_weights)

In [ ]:
from keras.optimizers import Adam, RMSprop

for layer in model.layers:
    layer.trainable = True

# model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-4, decay=0.0005), metrics=['accuracy', precision, recall, f1])
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=1e-4, decay=5e-4), 
              metrics=['categorical_accuracy', precision, recall, f1])
print('Model compiled')

In [ ]:
# Recompiling model AFTER running for x epochs.
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=5e-6, decay=5e-4), metrics=['accuracy', precision, recall, f1])
print('Model re-compiled')

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(filename, 
                             save_best_only=True, mode='max',
                             monitor='val_f1', verbose=1)
reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.05, mode='min', patience=2, verbose=1)
stopper = EarlyStopping(monitor='val_loss', mode='min', patience=3, verbose=1)

h = model.fit_generator(train_gen, 
                        steps_per_epoch=train_gen.samples//BATCH_SIZE + 1, 
                        validation_data=val_gen, 
                        validation_steps=val_gen.samples//BATCH_SIZE + 1, 
                        shuffle=True, 
                        epochs=10,
                        class_weight=class_weights, 
                        callbacks=[checkpoint, stopper, reducer])

fig = plt.figure()
plt.plot(h.history['loss'], 'r-')
plt.plot(h.history['val_loss'], 'b-')
plt.title('Loss plot')
plt.show()

fig = plt.figure()
plt.plot(h.history['f1'], 'r-')
plt.plot(h.history['val_f1'], 'b-')
plt.title('F1 plot')
plt.show()

fig = plt.figure()
plt.plot(h.history['precision'], 'r-')
plt.plot(h.history['val_precision'], 'b-')
plt.title('Precision plot')
plt.show()

fig = plt.figure()
plt.plot(h.history['recall'], 'r-')
plt.plot(h.history['val_recall'], 'b-')
plt.title('Recall plot')
plt.show()

In [ ]:
from keras.models import load_model

m = load_model('df_vasc_others_inceptionv3_512_full_train.h5', 
               compile=True, 
               custom_objects={'f1':f1, 
                               'precision':precision, 
                               'recall':recall})
print('Best model loaded')

In [ ]:
# y_pred = model.predict_generator(test_gen, verbose=1)
y_pred = m.predict_generator(test_gen, verbose=1)
# y_pred = model.predict_generator(pre_gen, verbose=1)

In [ ]:
y_true = test_gen.classes
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
from sklearn.metrics import classification_report, roc_curve, auc
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_true=y_true.ravel(), y_pred=y_pred.ravel()))

print(classification_report(y_true=y_true.ravel(), y_pred=y_pred.ravel()))

fpr, tpr, threshold = roc_curve(y_true, y_pred)
model_auc = auc(fpr, tpr)

print('AUC: {}'.format(model_auc))

In [ ]:
from vis.visualization import visualize_saliency, overlay
from vis.utils import utils
from keras import activations

In [ ]:
path_to_images = '/home/ubuntu/Data/Skin/nv_non_nv/toy_train/'
path_to_models = '/home/ubuntu/Notebooks/Rahul/Models/Hierarchical/'

In [ ]:
images_nv = os.listdir(path_to_images + 'NV')
images_non_nv = os.listdir(path_to_images + 'NON_NV')

model_name = 'nv_non_nv_inception_holdout_test_hist.h5' # Model here
layer_idx = -1 # Last prediction layer

In [ ]:
model.layers[layer_idx].activation = activations.linear
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.layers[-1].get_config())